<a href="https://colab.research.google.com/github/anminhyeok032/CJ_Object_Detection/blob/main/CJProject%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CJ 프로젝트
## CJ대한통운 미래기술챌린지 2023
#### 미래기술챌린지 2023 , 2. 비전을 활용한 스마트 검수
#### 사물 인식 프로그램

마지막 수정일자 : 2023.8.9

In [37]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Gpu 사용되는지 확인
!nvidia-smi

Mon Aug 14 09:57:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [39]:
# 기존 학습 데이터 ( test-15 (x3) )
# roboflow 가 프로젝트 generate가 안될시 새로 파서 병합하기
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="oF44mtd4eZpMYEOulCOi")
project = rf.workspace("cjproject").project("test-cryu5")
dataset = project.version(15).download("yolov7")

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 77, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_

KeyboardInterrupt: ignored

In [ ]:
# yolov7 필수 요소 다운로드
!git clone https://github.com/SkalskiP/yolov7.git
%cd yolov7
!git checkout fix/problems_associated_with_the_latest_versions_of_pytorch_and_numpy
!pip install -r requirements.txt

In [ ]:
# 드라이브에서 pt파일 yolov5 폴더로 이동
# 드라이브 경로가 다를시 수정 요망
!cp /content/drive/MyDrive/cj_data/0813_best_yolov7_xl.pt /content/yolov7

In [ ]:
# 엑셀 파일들 드라이브에서 끌어오기
# 직접 올린다면 굳이 실행하지 않아도 된다.
!cp /content/drive/MyDrive/cj_data/iou_names.xlsx /content/yolov7
!cp /content/drive/MyDrive/cj_data/labels.xlsx /content/yolov7
!cp /content/drive/MyDrive/cj_data/iou_index.xlsx /content/yolov7

In [ ]:
# detect 파일 github에서 끌어오기
# github가 public 상태 일때만 끌어올 수 있다.
%cd /content
!git clone https://github.com/anminhyeok032/CJ_Object_Detection.git
!cp /content/CJ_Object_Detection/detect.py /content/yolov7

In [ ]:
# data.yaml 파일 클래스 수 확인
%cat /content/test-15/data.yaml

In [ ]:
%cd /
# 추가 학습시에 val 설정 다시 할것
from glob import glob

train_img_list = glob('/content/test-15/train/images/*.jpg')
val_img_list = glob('/content/test-15/train/images/*.jpg')

print(len(train_img_list), len(val_img_list))

In [ ]:
# 추가 학습시에 val 설정 다시 할것
with open('/content/test-15/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/test-15/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
# 추가 학습시에 val 설정 다시 할것
import yaml

with open('/content/test-15/data.yaml', 'r') as f:
  data = yaml. safe_load(f)

print(data)

data['train'] = '/content/test-15/train.txt'
data['val'] = '/content/test-15/val.txt'

with open('/content/test-15/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

In [ ]:
# run this cell to begin training
# ***yolov7***
%cd /content/yolov7
!python train.py --batch 8 --epochs 55 --data /content/test-15/data.yaml --weights '0813_best_yolov7_xl.pt' --device 0 --cache --freeze 105


In [ ]:
# tensor 보드를 통해 학습 수준 확인
%load_ext tensorboard
%tensorboard --logdir /content/yolov7/runs/

In [ ]:
# 기존 train 데이터셋 확인
cnt = 0
for i in train_img_list :
  print(cnt,i+'\n')
  cnt = cnt+1

In [ ]:
# 한글 폰트 다운로드
!wget "https://www.wfonts.com/download/data/2016/06/13/malgun-gothic/malgun.ttf"
!mv malgun.ttf /usr/share/fonts/truetype/

In [41]:
# detect yolov7
%cd /content/yolov7
val_img_path = '/content/test-15/train/images/cart_0802_jpg.rf.14567b11bac23c59da95b1d09b78225a.jpg'
# val_img_path = '/content/test-15/train/images/13579-24218-sampleM_jpg.rf.2ca4df68416e8fd3a23fd14066fc8e81.jpg'
!python detect.py --weights /content/yolov7/runs/train/exp8/weights/best.pt --img-size 640 --conf 0.1 --source "{val_img_path}" --agnostic-nms

/content/yolov7
Namespace(weights=['/content/yolov7/runs/train/exp8/weights/best.pt'], source='/content/test-15/train/images/cart_0802_jpg.rf.14567b11bac23c59da95b1d09b78225a.jpg', img_size=640, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=True, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 b2a7de9 torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 354 layers, 71453050 parameters, 0 gradients, 190.1 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

[['107', '101'